In [1]:
!pip install transformers datasets peft trl bitsandbytes accelerate
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [2]:
pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import wandb
from transformers.integrations import WandbCallback
import sys



def get_train_val_ds():
    dataset = load_dataset("OpenAssistant/oasst1")
    dataset = dataset.shuffle().select(range(12500))

    train_dataset = dataset["train"]
    val_dataset = dataset["validation"]

    train_df = train_dataset.to_pandas()
    val_df = val_dataset.to_pandas()

    train_ds = preprocess_data(train_df)
    val_ds = preprocess_data(val_df)

    hf_train_ds = Dataset.from_pandas(train_ds)
    hf_val_ds = Dataset.from_pandas(val_ds)

    return hf_train_ds, hf_val_ds

def preprocess_data(df):
    def process_conversation(group):
        prompt_df = group[group["role"] == "prompter"]
        assistant_df = group[group["role"] == "assistant"]

        prompts = prompt_df["text"].tolist()
        responses = assistant_df.groupby("parent_id")["text"].apply(" ".join).tolist()

        if not prompts or len(prompts) != len(responses):
            return pd.DataFrame(columns=["prompt", "response"])

        base_prompt = prompts[0]
        augmented_prompts = [base_prompt] + [f"{base_prompt} {prompt}" for prompt in prompts[1:]]

        return pd.DataFrame({"prompt": augmented_prompts, "response": responses})

    processed_df = df.groupby("message_tree_id").apply(process_conversation).reset_index(drop=True)
    return processed_df[processed_df["response"] != ""]

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
train_ds, val_ds = get_train_val_ds()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

(…)-00000-of-00001-b42a775f407cee45.parquet:   0%|          | 0.00/39.5M [00:00<?, ?B/s]

(…)-00000-of-00001-134b8fd0c89408b6.parquet:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

In [5]:
train_ds

Dataset({
    features: ['prompt', 'response'],
    num_rows: 7501
})

In [6]:
import datasets
from peft import LoraConfig, get_peft_model
import torch
import transformers
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from dataclasses import dataclass
from typing import Optional

# Configuration classes
@dataclass
class BaseConfig:
    model_name: str = "microsoft/Phi-3-mini-4k-instruct"
    dataset_name: str = "OpenAssistant/oasst1"

training_config = {
    "bf16": True,
    "do_eval": False,
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 1,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 4,
    "per_device_train_batch_size": 4,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    "report_to":"wandb"
    }

peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": "all-linear",
    "modules_to_save": None,
}
@dataclass
class ModelConfig:
    torch_dtype: torch.dtype = torch.bfloat16
    trust_remote_code: bool = True
    use_cache: bool = False
    attn_implementation: str = "eager"

@dataclass
class BnbConfig:
    load_in_4bit: bool = True
    bnb_4bit_compute_dtype: torch.dtype = torch.float16
    bnb_4bit_quant_type: str = "nf4"
    bnb_4bit_use_double_quant: bool = True

# Helper functions
def create_bnb_config() -> BitsAndBytesConfig:
    return BitsAndBytesConfig(
        load_in_4bit=BnbConfig.load_in_4bit,
        bnb_4bit_compute_dtype=BnbConfig.bnb_4bit_compute_dtype,
        bnb_4bit_quant_type=BnbConfig.bnb_4bit_quant_type,
        bnb_4bit_use_double_quant=BnbConfig.bnb_4bit_use_double_quant
    )

def create_lora_config() -> LoraConfig:
    return LoraConfig(
        **peft_config
    )

def create_training_args() -> TrainingArguments:
    return TrainingArguments(
        **training_config
    )

In [7]:
wandb.init(project="phi-3-fine-tuning", name="qloratest")

train_arg = create_training_args()
lora_config = create_lora_config()
bnb_config = create_bnb_config()

In [8]:
log_level = train_arg.get_process_log_level()
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    BaseConfig.model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(BaseConfig.model_name, trust_remote_code=True)
tokenizer.model_max_length = 2048
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

[INFO|configuration_utils.py:733] 2024-09-17 13:35:41,473 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json


configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

[WARNING|dynamic_module_utils.py:383] 2024-09-17 13:35:42,007 >> A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
[INFO|configuration_utils.py:733] 2024-09-17 13:35:42,010 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json
[INFO|configuration_utils.py:800] 2024-09-17 13:35:42,013 >> Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-

modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2269] 2024-09-17 13:36:15,850 >> loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/tokenizer.model
[INFO|tokenization_utils_base.py:2269] 2024-09-17 13:36:15,851 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/tokenizer.json
[INFO|tokenization_utils_base.py:2269] 2024-09-17 13:36:15,853 >> loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/added_tokens.json
[INFO|tokenization_utils_base.py:2269] 2024-09-17 13:36:15,854 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/special_tokens_map

In [10]:
# Function to format the conversation
def format_conversation(example):
    prompt = example['prompt']
    response = example['response']
    conversation = f"Human: {prompt}\n\nAssistant: {response}"
    return {"conversation": conversation}

# Apply the formatting to the dataset
formatted_train_ds = train_ds.map(format_conversation)
formatted_val_ds = val_ds.map(format_conversation)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["conversation"], truncation=True, padding="max_length", max_length=512)

tokenized_train_ds = formatted_train_ds.map(tokenize_function, batched=True)
tokenized_val_ds = formatted_val_ds.map(tokenize_function, batched=True)


Map:   0%|          | 0/7501 [00:00<?, ? examples/s]

Map:   0%|          | 0/398 [00:00<?, ? examples/s]

Map:   0%|          | 0/7501 [00:00<?, ? examples/s]

Map:   0%|          | 0/398 [00:00<?, ? examples/s]

In [11]:
tokenized_train_ds.features

{'prompt': Value(dtype='string', id=None),
 'response': Value(dtype='string', id=None),
 'conversation': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [12]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    peft_config=lora_config,
    dataset_text_field="conversation",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=train_arg,
    callbacks=[WandbCallback()]
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
[INFO|training_args.py:2100] 2024-09-17 13:36:19,699 >> PyTorch: setting up devices
[INFO|training_args.py:2100] 2024-09-17 13:36:19,756 >> PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
[WARNING|

Step,Training Loss
20,1.333100
40,1.331900
60,1.485500
80,1.364800
100,1.400200
120,1.445100
140,1.451000
160,1.397000
180,1.410100
200,1.314400


[INFO|trainer.py:3503] 2024-09-17 13:37:49,868 >> Saving model checkpoint to ./checkpoint_dir/checkpoint-100
[INFO|configuration_utils.py:733] 2024-09-17 13:37:50,382 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json
[INFO|configuration_utils.py:800] 2024-09-17 13:37:50,385 >> Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
 

TrainOutput(global_step=1876, training_loss=1.2948236617960656, metrics={'train_runtime': 1657.6795, 'train_samples_per_second': 4.525, 'train_steps_per_second': 1.132, 'total_flos': 8.63595525266473e+16, 'train_loss': 1.2948236617960656, 'epoch': 1.0})

In [13]:
trainer.evaluate()

[INFO|trainer.py:811] 2024-09-17 14:03:58,888 >> The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt, response, conversation. If prompt, response, conversation are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
[INFO|trainer.py:3819] 2024-09-17 14:03:58,893 >> 
***** Running Evaluation *****
[INFO|trainer.py:3821] 2024-09-17 14:03:58,894 >>   Num examples = 398
[INFO|trainer.py:3824] 2024-09-17 14:03:58,895 >>   Batch size = 4


{'eval_loss': 1.3421905040740967,
 'eval_runtime': 25.9246,
 'eval_samples_per_second': 15.352,
 'eval_steps_per_second': 3.857,
 'epoch': 1.0}

In [14]:
from transformers import pipeline

test_input = "Human: What is an LLM in AI?\n\nAssistant:"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(test_input)
print(result[0]['generated_text'])

[WARNING|tokenization_utils_base.py:2857] 2024-09-17 14:04:25,209 >> Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
[WARNING|logging.py:328] 2024-09-17 14:04:25,404 >> The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


RuntimeError: expected mat1 and mat2 to have the same dtype, but got: float != c10::BFloat16

In [ ]:
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
wandb.finish()